

















# Data Transformation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd
import h5py
import psycopg2
import psycopg2.extras
from graphics import *
from datetime import timedelta, datetime, date
from scipy import spatial
import tables
import time
from time import sleep
from scipy.ndimage.filters import gaussian_filter1d

In [ ]:
sns.set()

In [ ]:
connect_str = """dbname='beesbook' user='mehmed' host='localhost' password='bbvision' application_name='memel'"""

In [ ]:
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    angle_360 = np.rad2deg((ang1 - ang2) % (2 * np.pi))
    if angle_360 <= 180:
        return angle_360
    else:
        return -1*(360 - angle_360)

vecA = np.array([-1,0])
vecB = np.array([2,1])
print(angle_between(vecA, vecB))
print(np.rad2deg(np.arctan2(*vecB[::-1])))

def angle_between_rad_vec(rad, vec):
    ang1 = rad
    ang2 = np.arctan2(*vec[::-1])
    angle_360 = np.rad2deg((ang1 - ang2) % (2 * np.pi))
    if angle_360 <= 180:
        return np.deg2rad(angle_360)
    else:
        return np.deg2rad(-1*(360 - angle_360))

print("anglebetweenradvec", angle_between_rad_vec(1, vecA))

    
def angle_between_rad(ang1, ang2):
    # returns diffrence in radian
    angle_360 = np.rad2deg((ang1 - ang2) % (2 * np.pi))
    if angle_360 <= 180:
        return np.deg2rad(angle_360)
    else:
        return np.deg2rad(-1*(360 - angle_360))
    
def get_bee_2D_vision(posX, posY, orientation, neighbour_positions, max_visual_range, num_bins = 36):
    """
    Seperate environment of an agent into bins and find nearest neighbour for all bins.
    """
    bins = np.zeros(num_bins)
    borders = np.linspace(-180, 180, num_bins + 1)
    sight_vectors = [neighbour_pos[:2] - np.array([posX, posY]) for neighbour_pos in neighbour_positions]
    angles = [[angle_between_rad_vec(orientation, sight_vector), sight_vector] for sight_vector in sight_vectors]
    for angle in angles:
        if (np.linalg.norm(angle[1])!=0):
            for i in range(len(borders)-1):
                if angle[0] <= borders[i+1]:
                    bins[i] = max([bins[i], (max_visual_range - np.linalg.norm(angle[1]))/max_visual_range]) 
                    break

                    return bins

In [ ]:
def normalize(vec, norm):
    """Normalizes the vector and multiplies it with norm."""
    return(norm*vec/np.linalg.norm(vec))


def turn_vector(vec, angle):
    x, y = vec
    x_new = x*np.cos(angle) - y*np.sin(angle)
    y_new = x*np.sin(angle) + y*np.cos(angle)
    vec_new = np.array((x_new, y_new))
    return vec_new

def get_wall_intersection(position, direction, borders):
    s = np.inf
    # make numpy arrays
    if type(position) == list:
        position = np.array(position)
    if type(direction) == list:
        direction = np.array(direction)
    if type(borders) == list:
        borders = np.array(borders)
    for point in borders:
        s_tmp = (point[0] - position[0])/direction[0]
        if s_tmp < 0:
            s_tmp = np.inf
        s = min(s, s_tmp)
        s_tmp = (point[1] - position[1]) / direction[1]
        if s_tmp < 0:
            s_tmp = np.inf
        s = min(s, s_tmp)
    intersection_point = position + s * direction
    return intersection_point

def get_walls_in_sight(position, orientation, radius, num_bins = 16, borders=[[0,0], [370, 370]]):
    """
    Ray tracing in order to detect walls.
    """
    bins = np.zeros(num_bins)
    bin_distance = 360/num_bins
    angles = np.linspace(-180+bin_distance/2, 180-bin_distance/2, num_bins, endpoint=True)
    angles = np.radians(angles)
    osin = np.sin(orientation)
    ocos = np.cos(orientation)
    target_dirs = [normalize(turn_vector(np.array([osin, ocos]), angle), radius) for angle in angles]
    for i, direction in enumerate(target_dirs):
        intersection_point = get_wall_intersection(position, direction, borders)
        bins[i] = max(0, 1 - np.linalg.norm(position-intersection_point)/radius)
    return bins

In [ ]:
class OneStepBeeHistory:
    
    # if bee_id's were unique per timestamp it would make much more sense 
    #to use bee_ids, but that is currently not the case
    def __init__(self, numberOfFeatures, track_ids):
        self.history = np.ones((2, len(track_ids), numberOfFeatures)) * (-1)
        self.bee_indicies = {}
        for i, key in enumerate(track_ids):
            self.bee_indicies[int(key)] = i
            
    def getKeys(self):
        return self.bee_indicies.keys()

    def save(self, track_id, bee):
        bee_index = self.bee_indicies[int(track_id)]
        self.history[0, bee_index, :] = self.history[1, bee_index, :]
        self.history[1, bee_index, :] = bee[:]
        
    def getMovement(self, track_id):
        bee_index = self.bee_indicies[track_id]
        if (self.history[0, bee_index, 3] == -1):
            # no history before this timestamp
            return 0, 0
        dx = self.history[1, bee_index, 0] - self.history[0, bee_index, 0]
        dy = self.history[1, bee_index, 1] - self.history[0, bee_index, 1]
        return dx, dy 
    
    def getOrientation(self, track_id):
        bee_index = self.bee_indicies[track_id]
        if (self.history[0, bee_index, 3] == -1):
            # no history before this timestamp
            return 0
        do = angle_between_rad(self.history[1, bee_index, 2], self.history[0, bee_index, 2])
        return do
       

    def getActivity(self, track_id, verbose = True):
        bee_index = self.bee_indicies[track_id]
        if (self.history[0, bee_index, 3] == -1):
            # no history before this timestamp
            return 0
        x0 = self.history[0, bee_index, 0]
        y0 = self.history[0, bee_index, 1]
        x1 = self.history[1, bee_index, 0]
        y1 = self.history[1, bee_index, 1]
        vectorLength = np.sqrt((x1-x0)*(x1-x0) + (y1-y0)*(y1-y0))
        dt = self.history[1, bee_index, 3] - self.history[0, bee_index, 3]
        if (dt < 1/3):
            dt = 0.3333333
            #print("UNSOUND DATA")
            #return 0
        if (verbose and vectorLength/(dt) > 100):
            print("INFOS:")
            print("timestep: ", self.history[0, bee_index, 3])
            print("bee id: ", self.history[0, bee_index, 5])
            print("vectorLength", vectorLength)
            print("avrg speed: ", vectorLength/(dt))
            print("dt: ", dt)
            print("x0, y0: ", x0, ", ", y0)
            print("x1, y1: ", x1, ", ", y1)
        return vectorLength/(dt)
    

In [ ]:
def getBeeSurroundings(currBee, neighbour_positions, history, max_visual_range, num_bins = 36, verbose = True, borders=[[0,0], [370, 370]]):
    """
    Seperate environment of an agent into bins and find nearest neighbour for all bins.
    Add orientation and activity of the neighbour bees.
    """
    posX = currBee[0]
    posY = currBee[1]
    orientation = currBee[2]
    bins_distance = np.zeros(num_bins)
    bins_SINorientation = np.zeros(num_bins)
    bins_COSorientation = np.zeros(num_bins)
    bins_activity = np.zeros(num_bins)
    bins_binary = np.zeros(num_bins)
    bins_relativeOsin = np.zeros(num_bins)
    bins_relativeOcos= np.zeros(num_bins)
    borders = get_walls_in_sight(np.array([posX, posY]), orientation, max_visual_range, num_bins, borders)
    angle_borders = np.linspace(-180, 180, num_bins + 1)
    sight_vectors = [[neighbour_pos[:2] - np.array([posX, posY]), neighbour_pos ]for neighbour_pos in neighbour_positions]
    angles = [[angle_between_rad_vec(orientation, sight_vector[0]), sight_vector[0], sight_vector[1]] for sight_vector in sight_vectors]
    for angle in angles:
        nbee_angle = angle[0] # based on this angle do bin seperation
        sight_vector = angle[1] # the sight vector
        nbee = angle[2] # neighbour bee
        if (np.linalg.norm(sight_vector)!=0):
            for i in range(len(angle_borders)-1):
                if nbee_angle <= angle_borders[i+1]:
                    if (bins_distance[i] < ((max_visual_range - np.linalg.norm(sight_vector))/max_visual_range)):
                        bins_distance[i] = (max_visual_range - np.linalg.norm(sight_vector))/max_visual_range
                        oBeeOrientation = nbee[2]
                        # sin and cos of angle 
                        od = orientation #angle_between_rad(orientation, oBeeOrientation)
                        bins_SINorientation[i] = np.sin(od)
                        bins_COSorientation[i] = np.cos(od)
                        track_id = nbee[4]
                        # TODO: check this
                        activity = history.getActivity(track_id, verbose)
                        bins_activity[i] = activity
                        bins_binary[i] = 1
                        # take vector that would look at me
                        # look how much sight vector is away from it
                        mOMrad = np.arctan2(currBee[0] - nbee[0], currBee[1] - nbee[1])
                        relativeO = angle_between_rad(oBeeOrientation, mOMrad)
                        bins_relativeOsin[i] = np.sin(relativeO)
                        bins_relativeOcos[i] = np.cos(relativeO)
                    break
    bins_distance = gaussian_filter1d(bins_distance, 0.4, mode='wrap')
    bins_activity = gaussian_filter1d(bins_activity, 0.4, mode='wrap')
    bins_binary = gaussian_filter1d(bins_binary, 0.4, mode='wrap')
    bins = np.concatenate((bins_distance, bins_SINorientation, bins_COSorientation, bins_activity, bins_binary, bins_relativeOsin, bins_relativeOcos, borders), axis = 0)
    #bins = np.append(np.append(np.append(np.append(np.append(np.append(np.append(bins_distance, bins_SINorientation), bins_COSorientation), bins_activity), bins_binary), bins_relativeOsin), bins_relativeOcos), borders)
    return bins

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
def transformCordinateSystem(o, dx, dy):
    x = np.cos(o) * dx - np.sin(o) * dy
    y = np.sin(o) * dx + np.cos(o) * dy
    return x, y    

In [ ]:
def getLength(x, y):
    return np.sqrt(x*x + y*y)

In [ ]:
def getData(querystring):
    start = time.time()
    with psycopg2.connect(connect_str) as conn:
        with conn.cursor('db_cursor', cursor_factory=psycopg2.extras.DictCursor) as cur:
            cur.execute(querystring)          
            data = cur.fetchall()
    print(len(data))
    end = time.time()
    print("Time in seconds: ", end - start)

    data = np.asarray(data)
    print("Data shape: ", data.shape)
    return data


In [ ]:
# starts[i] and stops[i] (starts[i] < stops[i]) should be intervals that are used for training generation. 
# large intervals should be avoided, because RAM might get filled 
# 2 Hour window with 16 bins -> 16 gigs needed 
starts = np.array(['2016-08-24 00:00:00', '2016-08-24 04:00:00', '2016-08-24 06:00:00'
                  , '2016-08-24 08:00:00', '2016-08-24 12:00:00', '2016-08-24 14:00:00'
                  , '2016-08-24 18:00:00']) 
stops  = np.array(['2016-08-24 01:00:00', '2016-08-24 05:00:00', '2016-08-24 07:00:00'
                  , '2016-08-24 09:00:00', '2016-08-24 13:00:00', '2016-08-24 15:00:00'
                  , '2016-08-24 19:00:00'])

In [ ]:
import datetime
import bb_utils.meta as bbmeta
import bb_utils.ids as bbids
import pandas as pd

BeeMeta = bbmeta.BeeMetaInfo()
max_visual_range = 20
bee_vision_bins = 16
bins_size = bee_vision_bins*8
num_features = bins_size+ 6
save_file_path = 'pathoffile.h5'
time_length = 40 #35
img_dtype = tables.Float64Atom()  # dtype in which the images will be saved
data_shape = (0, time_length - 1, bins_size+5 + 6 + 4)
hdf5_file = tables.open_file(save_file_path, mode='w')
hdf5_file.create_earray(hdf5_file.root, 'train', img_dtype, shape = data_shape)
hdf5_file.close()
track_movs = []
agenulls = 0
for i in range(0, len(starts)):
    print("Interval: ", i, "/", len(starts))
    start = '\'' + starts[i] + '\''
    stop = '\'' + stops[i] + '\''
    limit = 300000000
    cam_id = 2
    sqlquery = "SELECT timestamp, frame_id, track_id, bee_id, x_pos_hive, y_pos_hive, orientation_hive FROM bb_detections_2016_stitched WHERE timestamp >  TIMESTAMP %s AND timestamp < TIMESTAMP %s AND cam_id = %d ORDER BY timestamp ASC LIMIT %d;" % (start, stop, cam_id, limit)
    data = getData(sqlquery)
    #limit = 3000000
    sqlquery = "SELECT DISTINCT track_id FROM bb_detections_2016_stitched WHERE timestamp > TIMESTAMP %s AND timestamp < TIMESTAMP %s AND cam_id = %d ORDER BY track_id ASC LIMIT %d;" % (start, stop, cam_id, limit)
    track_ids = np.asarray(getData(sqlquery)).flatten().tolist()
    xmax = 0.0
    xmin = 1000.0
    ymax = 0.0
    ymin = 1000.0
    for i,d in enumerate(data):
        data[i, 0] = d[0].timestamp()
        xmax = np.maximum(xmax, d[4])
        xmin = np.minimum(xmin, d[4])
        ymax = np.maximum(ymax, d[4])
        ymin = np.minimum(ymin, d[4]) 
    print("xmax", xmax)
    print("xmin", xmin)
    print("ymax", ymax)
    print("ymin", ymin)
    borders=[[xmin-5,ymin-5], [xmax+5, ymax+5]]
    df = pd.DataFrame(data, columns=['datetime', 'frame_id', 'track_id', 'bee_id', 'x', 'y', 'orientation'])
    data = 0
    history = OneStepBeeHistory(num_features, track_ids)
    data_visual = []
    start = time.time()
    for i, tracks_t in enumerate(df.groupby('datetime')):
        if i % 250 == 0 and i > 1:
            print('Train data: {}/{}'.format(i, len(df.groupby('datetime'))))
        dct = tracks_t[1]
        positions = np.stack([dct.x, dct.y, dct.orientation, dct.datetime, dct.track_id, dct.bee_id], axis=1)
        positions = np.append(positions, np.zeros((len(dct), bins_size)), axis=1)
        neighbours= spatial.cKDTree(positions[:, :2]).query_ball_point(positions[:, :2], max_visual_range)
        for j, pos in enumerate(positions):
            history.save(pos[4], pos)
        for j, pos in enumerate(positions):
            if len(neighbours[j]) > 1:
                # Filter out myself from neighbours
                cur_neighbours = [x for x in neighbours[j] if x != j]
                neighbour_positions = positions[cur_neighbours]
                positions[j, 6:(num_features)] = getBeeSurroundings(pos, neighbour_positions, history, max_visual_range, num_bins = bee_vision_bins, borders = borders)
        data_visual.append(positions)
    data_visual = np.concatenate([x for x in data_visual])
    end = time.time()
    print("Time in seconds for Raycasting: ", end - start)
    print(data_visual.shape)
    df = pd.DataFrame(data_visual, columns=['x', 'y', 'orientation', 'datetime', 'track_id', 'bee_id'] + ['bee_vision_{}'.format(i) for i in range(bins_size)])
    tracks = []
    vision_bin_names = ['bee_vision_{}'.format(i) for i in range(bins_size)]
    start = time.time()
    for track in df.groupby('track_id'):
        track = track[1].sort_values('datetime')
        if len(track) > time_length:
            orientation_diff_sin = list()
            orientation_diff_cos = list()
            track_dx = list()
            track_dy = list()
            track_dt = list()
            tstep = datetime.datetime.utcfromtimestamp(track.datetime.values[0])
            bee_id = bbids.BeesbookID(bbids.BeesbookID._dec_to_bin(track.bee_id.values[0]))
            age = BeeMeta.get_age(bee_id, tstep).days + 1
            if pd.isnull(age) or age < 1:
                agenulls = agenulls + 1
                continue
            for i in range(1, len(track)):
                value =  track.orientation.values[i] 
                orientation_diff_sin.append(np.sin(value))
                orientation_diff_cos.append(np.cos(value))
                difx =  track.x.values[i] - track.x.values[i-1]
                dify =  track.y.values[i] - track.y.values[i-1]
                track_dx.append(difx) #(dxt)
                track_dy.append(dify) #(dyt)
                dtime = (track.datetime.values[i]-track.datetime.values[i-1])
                track_dt.append(dtime)
            stacklen = track.x.values[1:].shape
            values = np.stack([track.x.values[1:], track.y.values[1:], track.orientation.values[1:],
                               track.datetime.values[1:], track.track_id.values[1:], track.bee_id.values[1:],
                               np.ones(len(track_dt)) * age, np.zeros(stacklen), np.zeros(stacklen), np.zeros(stacklen),
                               track_dt, track_dx, track_dy, orientation_diff_cos, orientation_diff_sin], axis=1).astype(np.float64)        
            vision = track[vision_bin_names][1:].astype(np.float64)
            tracks.append(np.concatenate((values, vision), axis=1))
    end = time.time()
    print("Time in seconds for deltas: ", end - start)
    df = []
    track = 0
    vision = 0
    data_visual = 0
    start = time.time()
    with tables.open_file(save_file_path, mode='a') as hdf5_file:
        for track in tracks:
            maxdxdy = np.argmax(np.abs(track[:, 11:13]), axis = 0)
            if np.sum((np.linalg.norm(track[:, 11:13], axis= 1) > 25)) > 0:
                print("Big jump in training data. Value: ", track[maxdxdy[0], 7], track[maxdxdy[1], 8])
                print("Corrospending dts: ", track[maxdxdy[0], 6], track[maxdxdy[1], 6])
                continue
            dmov = np.sqrt(np.sum(np.abs(track[:, 7]))**2 + np.sum(np.abs(track[:, 8]))**2) / len(track)
            track_movs.append(dmov)
            for i in range(0, len(track)-time_length, 20):
                hdf5_file.root.train.append(track[None, i: i+time_length-1])
        end = time.time()
        print("Time in seconds: ", end - start)
        hdf5_file.close()
print("Age was this often zero: ", agenulls)
print("FINISHED")

In [ ]:
def findFloats(listOfFloats, value):
    for i, number in enumerate(listOfFloats):
        if np.abs(float(number)-float(value)) < 0.00001:
            return True
    return False

In [ ]:
# Random shuffle the training data on the hard disk, for large data sets, this takes ages
import random
import time
import tables
import numpy as np
start = time.time()
hdf5_file = tables.open_file('pathoffile.h5', mode='a')
print(hdf5_file.root.train.shape)
print(hdf5_file.root.train[0:10, :, :])
random.shuffle(hdf5_file.root.train)
print(hdf5_file.root.train.shape)
print(hdf5_file.root.train[0:10, :, :])
hdf5_file.close()
end = time.time()
hdf5_file.close()
print("Time in seconds for shuffle: ", end - start)

# Network
This part just shows some plots from part of the data

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.autograd
import sklearn
import pandas as pd
from torch import nn
import seaborn as sns
import tables 
from time import sleep

from tqdm import tqdm_notebook
import tqdm

%matplotlib inline

In [ ]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
def iterate_data(data, batchsize, shuffle=False):
    if shuffle:
        indices = np.arange(data.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, data.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield data[excerpt, :, :]

In [ ]:
hdf5_file = tables.open_file('pathoffile.h5', mode='r')
train = hdf5_file.root.train[:10000]
#The data is randomly shuffled. 50000 samples is enough to have representitive graphs for the data. 

In [ ]:
hdf5_file.close()

In [ ]:
train.shape

In [ ]:
train[20,0,:]

In [ ]:
X = train[:, :-1, 10:].astype(np.float32)

In [ ]:
b_bins = 16
# TODO: If bin_size changes the indicies below are wrong

In [ ]:
binary_values = X[:, :, 69:85].flatten()
indicies = np.where(np.round(binary_values) == 1)
print(indicies)

In [ ]:
#dt
plt.hist(np.abs(X[:, :, 0]).flatten(), 20)
plt.yscale('log')

In [ ]:
#dx
fig = plt.hist(X[:, :, 1].flatten(), 50)
plt.yscale('log')

In [ ]:
#dy
fig = plt.hist(X[:, :, 2].flatten(), 50)
plt.yscale('log')

In [ ]:
#do
fig = plt.hist(np.arctan2(X[:, :, 3], X[:, :, 4]).flatten(), 100)
plt.yscale('log')

In [ ]:
#d sin
fig = plt.hist(X[:, :, 3].flatten(), 100)
plt.yscale('log')

In [ ]:
# cos
plt.hist(X[:, :, 4].flatten(), 20)
plt.yscale('log')

In [ ]:

plt.hist(X[:, :, 5].flatten(), 20)
plt.yscale('log')

In [ ]:
#Raycasting:

In [ ]:
# distances
fig = plt.hist(X[:, :, 5:21].flatten()[indicies], 100)
plt.yscale('log')

In [ ]:
# orientation in radian
fig = plt.hist(np.arctan2(X[:, :, 21:37].flatten()[indicies], X[:, :, 37:53].flatten()[indicies]).flatten(), 100)
plt.yscale('log')

In [ ]:
#sin
fig = plt.hist(X[:, :, 21:37].flatten()[indicies], 100)
plt.yscale('log')

In [ ]:
#cos
fig = plt.hist(X[:, :, 37:53].flatten()[indicies], 100)
plt.yscale('log')

In [ ]:
#activity
fig = plt.hist(X[:, :, 53:69].flatten()[indicies], 100)
plt.yscale('log')

In [ ]:
#binary
fig = plt.hist(X[:, :, 69:85].flatten(), 100)
plt.yscale('log')

In [ ]:
# orientation relative in radian
fig = plt.hist(np.arctan2(X[:, :, 85:101].flatten()[indicies], X[:, :, 101:117].flatten()[indicies]).flatten(), 100)
#plt.yscale('log')

In [ ]:
#orientation relative sin
fig = plt.hist(X[:, :, 85:101].flatten()[indicies], 100)

In [ ]:
#orientation relative cos
fig = plt.hist(X[:, :, 101:117].flatten()[indicies], 100)

In [ ]:
#borders
fig = plt.hist(X[:, :, 117:133].flatten(), 100)
plt.yscale('log')

# Layer Norm LSTM
von Ben

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)


class Reshape(torch.nn.Module):
    def __init__(self, outer_shape):
        super(Reshape, self).__init__()
        self.outer_shape = outer_shape
    def forward(self, x):
        return x.view(x.size(0), *self.outer_shape)


class LSTMWrapper(torch.nn.Module):
    def __init__(self, module):
        super(LSTMWrapper, self).__init__()
        self.lstm = module

    def forward(self, x):
        h_t = torch.rand(x.shape[1], self.lstm.hidden_size, dtype=x.dtype).to(x.device)
        h_c = torch.rand(x.shape[1], self.lstm.hidden_size, dtype=x.dtype).to(x.device)

        h_ts = []
        h_cs = []
        for i in range(x.shape[0]):
            h_t, h_c = self.lstm(x[i].contiguous(), (h_t, h_c))
            h_ts.append(h_t)
            h_cs.append(h_c)

        return torch.stack(h_ts), (torch.stack(h_ts), torch.stack(h_cs))


class LayerNorm1D(nn.Module):

    def __init__(self, num_outputs, eps=1e-5, affine=True):
        super(LayerNorm1D, self).__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(1, num_outputs))
        self.bias = nn.Parameter(torch.zeros(1, num_outputs))

    def forward(self, inputs):
        input_mean = inputs.mean(1, keepdim=True).expand_as(inputs)
        input_std = inputs.std(1, keepdim=True).expand_as(inputs)
        x = (inputs - input_mean) / (input_std + self.eps)
        return x * self.weight.expand_as(x) + self.bias.expand_as(x)


class LayerNormLSTMCell(nn.LSTMCell):

    def __init__(self, input_size, hidden_size, bias=True):
        super().__init__(input_size, hidden_size, bias)

        self.ln_ih = nn.LayerNorm(4 * hidden_size)
        self.ln_hh = nn.LayerNorm(4 * hidden_size)
        self.ln_ho = nn.LayerNorm(hidden_size)

    def forward(self, input, hidden=None):
        self.check_forward_input(input)
        if hidden is None:
            hx = input.new_zeros(input.size(0), self.hidden_size, requires_grad=False)
            cx = input.new_zeros(input.size(0), self.hidden_size, requires_grad=False)
        else:
            hx, cx = hidden
        self.check_forward_hidden(input, hx, '[0]')
        self.check_forward_hidden(input, cx, '[1]')

        gates = self.ln_ih(F.linear(input, self.weight_ih, self.bias_ih)) \
                 + self.ln_hh(F.linear(hx, self.weight_hh, self.bias_hh))
        i, f, o = gates[:, :(3 * self.hidden_size)].sigmoid().chunk(3, 1)
        g = gates[:, (3 * self.hidden_size):].tanh()

        cy = (f * cx) + (i * g)
        hy = o * self.ln_ho(cy).tanh()
        return hy, cy


class LayerNormLSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers=1, bias=True, bidirectional=False):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        num_directions = 2 if bidirectional else 1
        self.hidden0 = nn.ModuleList([
            LayerNormLSTMCell(input_size=(input_size if layer == 0 else hidden_size * num_directions),
                              hidden_size=hidden_size, bias=bias)
            for layer in range(num_layers)
        ])

        if self.bidirectional:
            self.hidden1 = nn.ModuleList([
                LayerNormLSTMCell(input_size=(input_size if layer == 0 else hidden_size * num_directions),
                                  hidden_size=hidden_size, bias=bias)
                for layer in range(num_layers)
            ])

    def forward(self, input, hidden=None):
        seq_len, batch_size, hidden_size = input.size()  # supports TxNxH only
        num_directions = 2 if self.bidirectional else 1
        if hidden is None:
            hx = input.new_zeros(self.num_layers * num_directions, batch_size, self.hidden_size, requires_grad=False)
            cx = input.new_zeros(self.num_layers * num_directions, batch_size, self.hidden_size, requires_grad=False)
        else:
            hx, cx = hidden

        ht = [[None, ] * (self.num_layers * num_directions)] * seq_len
        ct = [[None, ] * (self.num_layers * num_directions)] * seq_len

        if self.bidirectional:
            xs = input
            for l, (layer0, layer1) in enumerate(zip(self.hidden0, self.hidden1)):
                l0, l1 = 2 * l, 2 * l + 1
                h0, c0, h1, c1 = hx[l0], cx[l0], hx[l1], cx[l1]
                for t, (x0, x1) in enumerate(zip(xs, reversed(xs))):
                    ht[t][l0], ct[t][l0] = layer0(x0, (h0, c0))
                    h0, c0 = ht[t][l0], ct[t][l0]
                    t = seq_len - 1 - t
                    ht[t][l1], ct[t][l1] = layer1(x1, (h1, c1))
                    h1, c1 = ht[t][l1], ct[t][l1]
                xs = [torch.cat((h[l0], h[l1]), dim=1) for h in ht]
            y  = torch.stack(xs)
            hy = torch.stack(ht[-1])
            cy = torch.stack(ct[-1])
        else:
            h, c = hx, cx
            for t, x in enumerate(input):
                for l, layer in enumerate(self.hidden0):
                    ht[t][l], ct[t][l] = layer(x, (h[l], c[l]))
                    x = ht[t][l]
                h, c = ht[t], ct[t]
            y  = torch.stack([h[-1] for h in ht])
            hy = torch.stack(ht[-1])
            cy = torch.stack(ct[-1])

        return y, (hy, cy)

# Gaussian Mixture Model Code
von Ben

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch.autograd import Variable
from torch.distributions import Categorical
import numpy as np
import math

def log_sum_exp(x, dim=1):
    x_max, x_argmax = x.max(dim, keepdim=True)
    x_max_broadcast = x_max.expand(*x.size())
    return x_max + torch.log(
        torch.sum(torch.exp(x - x_max_broadcast), dim=dim, keepdim=True))

class MDN(nn.Module):
    """A mixture density network layer
    The input maps to the parameters of a MoG probability distribution, where
    each Gaussian has O dimensions and diagonal covariance.
    Arguments:
        in_features (int): the number of dimensions in the input
        out_features (int): the number of dimensions in the output
        num_gaussians (int): the number of Gaussians per output dimensions
    Input:
        minibatch (BxD): B is the batch size and D is the number of input
            dimensions.
    Output:
        (pi, sigma, mu) (BxG, BxGxO, BxGxO): B is the batch size, G is the
            number of Gaussians, and O is the number of dimensions for each
            Gaussian. Pi is a multinomial distribution of the Gaussians. Sigma
            is the standard deviation of each Gaussian. Mu is the mean of each
            Gaussian.
    """

    def __init__(self, in_features, out_features, num_gaussians):
        super(MDN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_gaussians = num_gaussians
        self.pi = nn.Sequential(
            nn.Linear(in_features, num_gaussians), nn.LogSoftmax(dim=1))
        self.sigma = nn.Linear(in_features, out_features * num_gaussians)
        self.mu = nn.Linear(in_features, out_features * num_gaussians)

    def forward(self, minibatch):
        pi = self.pi(minibatch)
        sigma = self.sigma(minibatch)
        # original sigma = torch.clamp(sigma, np.log(np.sqrt(1e-4)), 1e8)
        # working 
        #sigma =  torch.clamp(sigma, np.log(np.sqrt(1e-3)), 5e1)
        sigma = torch.clamp(sigma, np.log(np.sqrt(1e-4)), 1e4)
        sigma = sigma.view(-1, self.num_gaussians, self.out_features)
        mu = self.mu(minibatch)
        mu = mu.view(-1, self.num_gaussians, self.out_features)
        return pi, sigma, mu

    @staticmethod
    def gaussian_probability(sigma, x_mu, x):
        """Returns the probability of `data` given MoG parameters `sigma` and `mu`.
        Arguments:
            sigma (BxGxO): The standard deviation of the Gaussians. B is the batch
                size, G is the number of Gaussians, and O is the number of
                dimensions per Gaussian.
            mu (BxGxO): The means of the Gaussians. B is the batch size, G is the
                number of Gaussians, and O is the number of dimensions per Gaussian.
            data (BxI): A batch of data. B is the batch size and I is the number of
                input dimensions.
        Returns:
            probabilities (BxG): The probability of each point in the probability
                of the distribution in the corresponding sigma/mu index.
        """
        x = x.unsqueeze(1).expand_as(sigma)
        var = (torch.exp(sigma)**2)
        return -((x - x_mu)**2) / (2 * var + 1e-4) - sigma - math.log(
            math.sqrt(2 * math.pi))

    @staticmethod
    def mdn_loss(pi, sigma, mu, target):
        """Calculates the error, given the MoG parameters and the target
        The loss is the negative log likelihood of the data given the MoG
        parameters.
        """
        nll = log_sum_exp(pi[:, :, None] +
                          MDN.gaussian_probability(sigma, mu, target))
        nll = -torch.sum(nll, dim=-1)
        return torch.mean(nll)

    @staticmethod
    def sample(pi, sigma, mu):
        """Draw samples from a MoG.
        """
        categorical = Categorical(torch.exp(pi))
        pis = list(categorical.sample().data)
        sigma = torch.exp(sigma)
        sample = Variable(
            sigma.data.new(sigma.size(0), sigma.size(2)).normal_())
        for i, idx in enumerate(pis):
            sample[i] = sample[i].mul(sigma[i, idx]).add(mu[i, idx])
        return sample

## RNN

In [ ]:
def initialize_weights(model):
    if type(model) in [nn.Linear]:
        nn.init.xavier_normal_(model.weight.data)
    elif type(model) in [nn.LSTM, nn.RNN, nn.GRU]:
        nn.init.xavier_normal_(model.weight_hh_l0)
        nn.init.xavier_normal_(model.weight_ih_l0)

In [ ]:
import torch.autograd as autograd

class SimpleRNN(torch.nn.Module):
    def __init__(self, n_features, n_outputs):
        super(SimpleRNN, self).__init__()
        # 32 was used for all the simulated data
        #hidden_dim = 32 #
        
        hidden_dim = 128 #hidden_dim

        #self.inp = torch.nn.Linear(n_features, hidden_size)
        num_layers = 2
        #self.rnn = LayerNormLSTM(n_features, hidden_dim, num_layers = num_layers)
        self.rnn = torch.nn.LSTM(n_features, hidden_dim, num_layers = num_layers)
        
        # 64 was used for all the simulated data
        #self.out = torch.nn.Linear(hidden_dim, 64)
        
        #self.out = torch.nn.Linear(hidden_dim, 32)
        self.mdn = MDN(hidden_dim, n_outputs, 5)

        
        #self.hidden = None
        
        initialize_weights(self.rnn)
        #initialize_weights(self.out)
        initialize_weights(self.mdn)

    def step(self, inputs, hidden=None, verbose=False):
        #input = self.inp(input)
        if verbose:
            print("Step 0:")
            print(inputs.shape)
        inputs = inputs.permute([1, 0, 2])
        if verbose:
            print("Step 1:")
            print(inputs.shape)
        #self.rnn.flatten_parameters()
        output, hidden = self.rnn(inputs, hidden)
        output = output[-1, :, :] #output[:, :, :] #output[-1, :, :]
        #output = output.permute([1, 0, 2])
        if verbose:
            print("Step 3:")
            print(output.shape)
        output = output.squeeze()
        if verbose:
            print("Step 4:")
            print(output.shape)
        #output = self.out(output)
        if verbose:
            print("Step 5:")
            print(output.shape)
            print(output)
        output = self.mdn(output)
        return output, hidden

    def forward(self, inputs, hidden=None, verbose=False):
        if verbose:
            print("inputs size: ", inputs.size)
        batch_size = inputs.size(0)    
        output, hidden = self.step(inputs, hidden, verbose=verbose)
        return output, hidden

In [ ]:
import torch.autograd as autograd

class BeeRNN(torch.nn.Module):
    def __init__(self, n_features, n_outputs):
        super(BeeRNN, self).__init__()
        # 32 was used for all the simulated data
        #hidden_dim = 32 #
        
        hidden_dim = 32 #hidden_dim

        #self.inp = torch.nn.Linear(n_features, hidden_size)
        num_layers = 1
        #self.rnn = LayerNormLSTM(n_features, hidden_dim, num_layers = num_layers)
        self.rnn = torch.nn.LSTM(n_features, hidden_dim, num_layers = num_layers)
        self.out = torch.nn.Linear(hidden_dim, 64) # 64
        self.mdn = MDN(64, n_outputs, 5)
        
        initialize_weights(self.rnn)
        initialize_weights(self.out)
        initialize_weights(self.mdn)

    def step(self, inputs, hidden=None, verbose=False):
        #input = self.inp(input)
        if verbose:
            print("Step 0:")
            print(inputs.shape)
        inputs = inputs.permute([1, 0, 2])
        if verbose:
            print("Step 1:")
            print(inputs.shape)
        #self.rnn.flatten_parameters()
        output, hidden = self.rnn(inputs, hidden)
        output = output[-1, :, :] #output[:, :, :] #output[-1, :, :]
        #output = output.permute([1, 0, 2])
        if verbose:
            print("Step 3:")
            print(output.shape)
        output = output.squeeze()
        if verbose:
            print("Step 4:")
            print(output.shape)
        output = self.out(output)
        if verbose:
            print("Step 5:")
            print(output.shape)
            print(output)
        output = self.mdn(output)
        return output, hidden

    def forward(self, inputs, hidden=None, verbose=False):
        if verbose:
            print("inputs size: ", inputs.size)
        batch_size = inputs.size(0)    
        output, hidden = self.step(inputs, hidden, verbose=verbose)
        return output, hidden

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [ ]:
def mselosses(pi, mu, target):
    pis = torch.exp(pi)/ torch.sum(torch.exp(pi), dim = 1)[:, None].expand(-1, 1)
    Y = torch.sum((pis[:,:,None] * mu)[:, :, :], dim = 1)
    return torch.sum((Y[:, :2]-target[:, :2])**2)/target.shape[0]
    #return ((Y[:, :2]-target[:, :2])**2).mean()

In [ ]:
def anglediffs(pi, mu, target):
    pis = torch.exp(pi)/ torch.sum(torch.exp(pi), dim = 1)[:, None].expand(-1, 1)
    Y = torch.sum((pis[:,:,None] * mu)[:, :, :], dim = 1)
    n1 = torch.norm(Y[:, 2:], dim = 1)
    n2 = torch.norm(target[:, 2:], dim = 1) 
    cosa = (torch.sum(target[:, 2:] * Y[:, 2:], dim = 1)) / (n1 * n2)
    return torch.sum(torch.acos(cosa))/target.shape[0]

In [ ]:
def getMeanSample(pi, sigma, mu):
    pis = torch.exp(pi)/ torch.sum(torch.exp(pi), dim = 1)[:, None].expand(-1, 5)
    Y = torch.sum((pis[:,:,None] * mu)[:, :, :], dim = 1)
    return Y[:, :]
    #return ((Y[:, :2]-target[:, :2])**2).mean()

In [ ]:
vision_bins = 16

In [ ]:
constrnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
optimizer_const = torch.optim.RMSprop(constrnn.parameters(), lr=0.0001)
running_loss_const = 0.0


In [ ]:
rnn = SimpleRNN(n_features=5 +(vision_bins * 8), n_outputs=4).cuda()
optimizer = torch.optim.RMSprop(rnn.parameters(), lr=0.0001)
running_loss = 0.0


In [ ]:
rnn = BeeRNN(n_features=5 +(vision_bins * 8), n_outputs=4).cuda()
rnn = torch.load('beernn.pt')
optimizer = torch.optim.RMSprop(rnn.parameters(), lr=0.001)
running_loss = 0.0


In [ ]:
constrnn = BeeRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
optimizer_const = torch.optim.RMSprop(constrnn.parameters(), lr=0.001)
running_loss_const = 0.0

In [ ]:
optimizer = torch.optim.RMSprop(rnn.parameters(), lr=0.00001)

In [ ]:
rec_losses = []  
rec_losses_const = []
mse_diffs = []
mse_diffs_const = []

In [ ]:
batch_X = [] 
batch_Y = [] 
tqdm_iter = [] 
minibatch_idx = [] 
batch_data = []  
loss = []  
mse_diffs = []  
batch_X_const = []  
batch_Y_const = []  
rec_losses_test_const = []
rec_losses_test = []

In [ ]:
#Trains normal and constant network
import torch
from torch import autograd
from random import randint

# Training
tqdm_epochs = tqdm_notebook(list(range(3)))
rec_losses = []  
rec_losses_const = []
batch_size = 32
loss_step = 100
i = 0
testinterval = 100#100

mse_diffs = []
mse_diffs_test = []
mse_diffs_const = []
test_rnn_const = []
test_rnn = []
h5ftrain = tables.open_file('pathtofile.h5', mode='r')
h5ftest = tables.open_file('pathtofile.h5', mode='r')
train = h5ftrain.root.train
test = h5ftest.root.train[:15000] 
print("trainshape: ", train.shape)
print("testshape: ", test.shape)
ci = 0
constant_stop = 200000000000000000000

print(train.shape)
print(test.shape)
for epoch in tqdm_epochs:    
    tqdm_iter = tqdm.tqdm_notebook(enumerate(iterate_data(train , batchsize=batch_size, shuffle=False)),
                                   total=train.shape[0] // batch_size)
    for minibatch_idx, (batch_data) in tqdm_iter: 
        batch_X = batch_data[:, :-1, 6+4:].astype(np.float32)
        i = i+1
        batch_Y = batch_data[:, -1, (1+6+4):(5+6+4)].astype(np.float32)
        with autograd.detect_anomaly():
            rnn.zero_grad()
            batch_X = torch.autograd.Variable(torch.from_numpy(batch_X)).cuda()
            batch_Y = torch.autograd.Variable(torch.from_numpy(batch_Y).float()).cuda()
            Y_predicted, hidden = rnn.forward(batch_X, verbose=False) #847
            pi, sigma, mu = Y_predicted
            loss = MDN.mdn_loss(pi, sigma, mu, batch_Y).cuda()
            loss.backward()
            optimizer.step()
            rec_losses.append(float(loss.cpu().data.numpy()))
            mse_diffs.append(mselosses(pi, mu, batch_Y).cpu().data.numpy())
            tqdm_epochs.set_description("Loss {:6.5f}".format( np.mean(rec_losses[-100:])))
        if i == testinterval:
            i = 0
            rec_losses_test_const = []
            rec_losses_test = []
            mse_diffs_testintervall = []
            for batch_data in iterate_data(test[:10000] , batchsize=batch_size, shuffle=False):
                rnn.zero_grad()
                batch_X = batch_data[:, :-1, 6+4:].astype(np.float32)
                batch_Y = batch_data[:, -1, (1+6+4):(5+6+4)].astype(np.float32)
                batch_X = torch.from_numpy(batch_X)
                batch_X = torch.autograd.Variable(batch_X).cuda()
                batch_X_const = torch.from_numpy(np.zeros(batch_X.shape).astype(np.float32))
                batch_X_const = torch.autograd.Variable(batch_X_const).cuda()
                batch_Y = torch.from_numpy(batch_Y)
                batch_Y = torch.autograd.Variable(batch_Y.float()).cuda()
                Y_predicted, hidden = rnn.forward(batch_X, verbose=False)
                pi, sigma, mu = Y_predicted
                loss = MDN.mdn_loss(pi, sigma, mu, batch_Y) 
                mse_diffs_testintervall.append(mselosses(pi, mu, batch_Y).cpu().data.numpy())
                rec_losses_test.append(np.mean(float(loss.cpu().data.numpy())))
            mse_diffs_test.append(np.mean(mse_diffs_testintervall))
            test_rnn.append(np.mean(rec_losses_test))


In [ ]:
h5ftrain.close()
h5ftest.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,4))
ax.plot(pd.Series(rec_losses).rolling(1).mean())

plt.xlabel('Training batches')
plt.ylabel('MDN Loss')
plt.xscale('log')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,4))
ax.plot(pd.Series(test_rnn).rolling(1).mean())
plt.xlabel('Test batches')
plt.ylabel('MDN Loss')
plt.xscale('log')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,4))
ax.plot(pd.Series(mse_diffs).rolling(1).mean())
plt.xlabel('Train batches')
plt.ylabel('MSE Loss')
plt.xscale('log')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,4))
ax.plot(pd.Series(mse_diffs_test).rolling(1).mean())
plt.xlabel('Test batches')
plt.ylabel('MSE Loss')
plt.xscale('log')
plt.show()

# Simulation

orientierung der Biene visualisieren 

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import seaborn as sns 
from graphics import *
import random

from datetime import timedelta, datetime, date
from scipy import spatial

from simulationhelpers import *

import time
import psycopg2
import psycopg2.extras

In [ ]:
sns.set()
connect_str = """dbname='beesbook' user='mehmed' host='localhost' password='bbvision' application_name='memel'"""

In [ ]:
start = time.time()
with psycopg2.connect(connect_str) as conn:
    with conn.cursor('db_cursor', cursor_factory=psycopg2.extras.DictCursor) as cur:
        cur.execute("""SELECT timestamp, track_id, bee_id, x_pos_hive, y_pos_hive, orientation_hive, cam_id
        FROM bb_detections_2016_stitched
        WHERE timestamp > TIMESTAMP '2016-07-20 14:00:00' AND timestamp < TIMESTAMP '2016-07-20 14:05:00'
        AND (cam_id = 0)
        ORDER BY timestamp ASC LIMIT %(limit)s;""", {'limit': 500000})          
        rows = cur.fetchall()
#AND (cam_id = 0 OR cam_id = 1)
print(len(rows))
end = time.time()
print("Time in seconds: ", end - start)


data = np.asarray(rows)
rows = 0
print("Data shape: ", data.shape)

for i,d in enumerate(data):
    data[i, 0] = d[0].timestamp()
print(data.shape)



In [ ]:
df = pd.DataFrame(data, columns=['datetime', 'track_id', 'bee_id', 'x', 'y', 'orientation', 'cam_id'])

In [ ]:
start = time.time()
with psycopg2.connect(connect_str) as conn:
    with conn.cursor('db_cursor', cursor_factory=psycopg2.extras.DictCursor) as cur:
        cur.execute("""SELECT DISTINCT track_id
        FROM bb_detections_2016_stitched
        WHERE timestamp > TIMESTAMP '2016-07-20 14:00:00' AND timestamp < TIMESTAMP '2016-07-20 14:05:00'
        AND (cam_id = 0)
        ORDER BY track_id ASC LIMIT %(limit)s;""", {'limit': 500000})          
        track_ids = cur.fetchall()
#AND (cam_id = 0 OR cam_id = 1)
print(len(track_ids))
end = time.time()
print("Time in seconds: ", end - start)

In [ ]:
track_ids = np.asarray(track_ids).flatten().tolist()

In [ ]:
track_ids[:10]

In [ ]:
10906746571586957921

In [ ]:
# TODO: put the smallest value around 0, 0
num_agents = len(track_ids)
print("Number of agents: ", num_agents)
num_timestamps = df.groupby('datetime').nunique().shape[0]
print("Number of timesteps: ", num_timestamps)
num_features = 3
historys = np.ones(((len(track_ids), num_timestamps, num_features))) * (-1000)
timestamp = 0
for dataframes in df.groupby('datetime'):
    for index, row in dataframes[1].iterrows():
        bee_index = track_ids.index(row['track_id'])
        #oldX = historys[bee_index, timestamp-1,0]
        #oldY = historys[bee_index, timestamp-1,1]     
        x = row['x']
        y = row['y']
        o = row['orientation']
        #if (timestamp > 0) and ((oldX < 0 and oldY < 0) or ((abs(oldX-x) > 500) or (abs(oldY-y) > 500))):
        #    continue
        historys[bee_index, timestamp,:] = x, y, o
    timestamp = timestamp + 1



In [ ]:
historys.shape

In [ ]:
initarray = np.ones(((len(track_ids), num_features))) * (-1000)
a = 0
for i in range(0, historys.shape[0]):
    for j in range(0, num_timestamps):
        if historys[i, j, 0] > - 999:
            initarray[i, :] = historys[i, j, :]
            a = a + 1
            break
print(len(track_ids))
print(a)

In [ ]:
np.sum(initarray[:, 0] > -999)

In [ ]:
historys.shape

In [ ]:
historys[initarray[:, 0] > -999, :5, :]

In [ ]:
for df in initarray[initarray[:, 0] > -999]:
    print(df)

In [ ]:
agents.shape

In [ ]:
class Bee:
    def __init__(self, N, pos, color, velo, bee_id, vision_bins, o = 0):
        self.point = Circle(Point(pos[0],pos[1]),10)
        self.visionpoint = Circle(Point(pos[0],pos[1]),50)
        self.line = Line(self.point.getCenter(), self.point.getCenter())
        self.oldPos = np.array([pos[0], pos[1]])
        self.point.setFill(color)
        self.history = []
        self.velo = velo
        self.id = bee_id
        self.vision = np.zeros(vision_bins*8)
        self.t = 0
        self.orientation = o
        self.oldOrientation = o
        self.xO = np.cos(o)
        self.yO = np.sin(o)
        self.dO = 0
        self.gotMoved = False
        self.headingVec = Line(self.point.getCenter(), self.point.getCenter()) 
        self.vlines = []
        self.label = np.zeros((4))
        for i in range(0, vision_bins+1):
            self.vlines.append(Line(self.point.getCenter(), Point(0, 0)))
        self.wlines = []
        for i in range(0, vision_bins):
            self.wlines.append(Line(self.point.getCenter(), Point(0, 0))) 
        self.opinion = np.random.uniform(-1, 1)
        self.uncertainty = np.random.uniform(0, 0.5)
        self.conv = np.random.uniform(0.1, 0.4)
        self.future_vs = np.zeros(40)
        self.velospeed = 1
        self.neighbours = np.zeros(N) #TODO: 
        self.neighbourocc = []
        self.repulsion_time = 0

    def getX(self): 
        return self.point.getCenter().getX()

    def getY(self):
        return self.point.getCenter().getY()

    def getPos(self):
        return np.array([self.getX(), self.getY()])

    def setColor(self,color):
        self.point.setFill(color)
        
    def getT(self): 
        return self.t
    
    def incT(self): 
        self.t = self.t + 0.33
        return self.t
    
    def setOrientation(self, xO, yO):
        self.oldOrientation = self.orientation
        o = np.arctan2(xO, yO)
        self.dO = angle_between_rad(o, self.orientation)
        self.xO = xO
        self.yO = yO
        self.orientation = o
            
    def setOrientationO(self, o):
        self.oldOrientation = self.orientation
        self.dO = angle_between_rad(o, self.orientation)
        self.orientation = o
        self.xO = np.sin(o)
        self.yO = np.cos(o)
        
    def getCurrState(self, vision_bins):
        mo = angle_between_rad_vec(self.orientation, self.velo)
        #array = np.array([self.getX(), self.getY(), self.orientation, self.getT(), self.id, np.sin(mo), np.cos(mo)])
        array = np.array([self.getX(), self.getY(), self.orientation, self.getT(), self.id, np.sin(mo), np.cos(mo)])
        return np.concatenate([array, self.vision])

    
    def appendTimestep(self, OSHistory, vision_bins):
        dx, dy = self.velo[0], self.velo[1]
        dxt, dyt = transformCordinateSystem(self.oldOrientation, dx, dy)
        array = np.array([self.getX(), self.getY(), self.orientation, np.sum(self.neighbours > 0), self.velospeed, len(self.neighbourocc), 
                          0, dx, dy, np.cos(self.orientation), np.sin(self.orientation)])
        self.history.append(np.concatenate([np.concatenate([self.label, array]), self.vision]))


        
    def __str__(self):
        return "Bee ID = " + self.id.__str__() + "velo = " + self.velo.__str__()  + "; position = (" + self.point.getCenter().getX().__str__() + ", " + self.point.getCenter().getY().__str__() + ")" + "; orientation = (" + self.orientation.__str__()

def initialize(num_agents, window, winWidth, winHeight, vision_bins, draw, algorithm):
    global agents
    agents = []
    bee_id = 0
    c = getDistinctColors(num_agents)
    # Random init
    if algorithm == 0 or algorithm == 1 or algorithm == 2 or algorithm == 4 or algorithm == 5 or algorithm == 6:
        for i in range(num_agents):
            agents.append(Bee(num_agents, np.array([np.random.uniform(0, winWidth), np.random.uniform(0, winHeight)])
                              ,color_rgb(c[bee_id][0]
                                         ,c[bee_id][1],c[bee_id][2]), np.zeros(2), bee_id, vision_bins))
            bee_id = bee_id + 1
    elif algorithm == 9:
        for i in range(num_agents):
            agents.append(Bee(num_agents, np.array([initarray[bee_id, 0], initarray[bee_id, 1]]),color_rgb(c[bee_id][0],c[bee_id][1],c[bee_id][2]), np.zeros(2), bee_id, vision_bins))
            bee_id = bee_id + 1
    elif algorithm == 3:
        x = windows_data[0, 0] #winWidth/2
        y = windows_data[0, 1] #winHeight/2
        o = windows_data[0, 2]
        agents.append(Bee(num_agents, np.array([x, y]),color_rgb(c[bee_id][0],c[bee_id][1],c[bee_id][2]),
                            np.zeros(2), bee_id, vision_bins, o))
    else:
        for i in range(num_agents):
            agents.append(Bee(num_agents, np.array([initarray[bee_id, 0], initarray[bee_id, 1]]),color_rgb(c[bee_id][0],c[bee_id][1],c[bee_id][2]), np.zeros(2), bee_id, vision_bins))
            bee_id = bee_id + 1
    draw_agents(window, draw)

    
def draw_agents(window, draw):
    """Draw all agents into window"""
    if draw == True:
        for ai in agents:
            ai.point.undraw()
            ai.point.draw(window)
            n_distincts = min(np.sum(ai.neighbours > 0), 4)/4.0
            ai.point.setFill(color_rgb(255, int(round(255-(n_distincts * 255))),0))

                
def move_agent(aj, window, draw, algoirthm = -1):
    """Move agent according to agent.velo"""
    #Movement
    aj.oldPos = np.array([aj.getX(), aj.getY()])
    aj.point.move(aj.velo[0],aj.velo[1]) 
    if draw == True:
        aj.line.undraw() 
    aj.line = Line(aj.point.getCenter(), Point(aj.getX() + aj.yO, aj.getY() + aj.xO))
    if draw == True:
        if algorithm == 5 or algorithm == 6:
            n_distincts = min(np.sum(aj.neighbours > 0), 4)/4.0
            aj.point.setFill(color_rgb(255, int(round(255-(n_distincts * 255))),0))
        aj.line.setFill("black")
        aj.line.setArrow("last")
        aj.line.draw(window)
    aj.headingVec = Line(Point(aj.oldPos[0], aj.oldPos[1]), aj.point.getCenter())
    if draw == True:
        aj.headingVec.draw(window)

###################################### Movement Models ######################################
# takes the steps from historys
def real_step(aj, agents, window):
    new_pos = historys[aj.id, timestep, :]
    m = np.array([0, 0])
    aj.point.undraw()
    aj.gotMoved = False
    if new_pos[0] > -999 or new_pos[1] > -999:
        m = new_pos[:2] - [aj.getX(), aj.getY()]
        aj.setOrientation(np.cos(new_pos[2]), np.sin(new_pos[2]))
        aj.point.draw(window)
        aj.gotMoved = True
    aj.velo = m

    
# takes the steps from window_data
def window_step(aj):
    w = windows_data[timestep+1, 6:]
    Y = w[1:5]
    dx = Y[0]
    dy = Y[1]
    dxt, dyt = dx, dy
    aj.velo = np.array([dxt, dyt])  
    newOrientation = np.arctan2(Y[3], Y[2]) 
    if newOrientation > np.pi:
        newOrientation = - np.pi + (newOrientation - np.pi)
    if newOrientation < -np.pi:
        newOrientation =  np.pi - (- np.pi - newOrientation)
    aj.setOrientationO(newOrientation)  

    
# random walk    
def simple_step(aj, agents):
        m = np.array([(-1)**random.randint(0,1) * random.uniform(0,2.5), (-1)**random.randint(0,1) * random.uniform(0,2.5)])
        aj.velo = m
        o = np.arctan2(aj.velo[0], aj.velo[1])
        aj.setOrientationO(o)

# Super Simple Model
def constant_step(aj, agents, winWidth, winHeight, max_visual_range):
    label = np.zeros((4))
    posX = aj.point.getCenter().getX()
    posY = aj.point.getCenter().getY()
    m = aj.velo
    m = normalize(m, 1)
    o = np.arctan2(m[1], m[0])
    if posX - max_visual_range < 0:
        d = 0.0 - (posX - max_visual_range)
        wd = d/max_visual_range
        m[0] = m[0] + wd * 1
        label[3] = 1
    if posX + max_visual_range > winWidth:
        d = (posX + max_visual_range) - winWidth
        wd = d/max_visual_range
        m[0] = m[0] + wd * -1
        label[3] = 1
    if posY - max_visual_range < 0:
        d = 0.0 - (posY - max_visual_range)
        wd = d/max_visual_range
        m[1] = m[1] + wd * 1
        label[3] = 1
    if posY + max_visual_range > winHeight:
        d = (posY + max_visual_range) - winHeight
        wd = d/max_visual_range
        m[1] = m[1] + wd * -1
        label[3] = 1
    o = np.arctan2(m[1], m[0])
    aj.label = label
    return m, o
    
def complex_model(aj, agents, winWidth, winHeight, max_visual_range):
    posX = aj.point.getCenter().getX()
    posY = aj.point.getCenter().getY()
    label = np.zeros((4))
    w = 0.2
    ai, dist = find_nearest_neighbor(aj, agents)
    m = aj.velo
    max_visual_range = max_visual_range - 5
    if dist < 10:
        ma = np.array([posX - ai.getX(), posY - ai.getY()])
        m = m + 0.01 * ma
        label[2] = 1
    elif dist < max_visual_range:
        m = m + w*np.array([ai.velo[0], ai.velo[1]])
        label[1] = 1
    m = normalize(m, 1)
    if posX - max_visual_range < 0:
        d = 0.0 - (posX - max_visual_range)
        wd = d/max_visual_range
        m[0] = m[0] + wd * 1
        label[3] = 1
    if posX + max_visual_range > winWidth:
        d = (posX + max_visual_range) - winWidth
        wd = d/max_visual_range
        m[0] = m[0] + wd * -1
        label[3] = 1
    if posY - max_visual_range < 0:
        d = 0.0 - (posY - max_visual_range)
        wd = d/max_visual_range
        m[1] = m[1] + wd * 1
        label[3] = 1
    if posY + max_visual_range > winHeight:
        d = (posY + max_visual_range) - winHeight
        wd = d/max_visual_range
        m[1] = m[1] + wd * -1
        label[3] = 1
    if np.sum(label) <= 0:
        label[0] = 1
    o = np.arctan2(m[1], m[0])
    aj.label = label 
    return m, o



def social_model(aj, agents, winWidth, winHeight, max_visual_range):
    posX = aj.point.getCenter().getX()
    posY = aj.point.getCenter().getY()
    label = 0
    w = 0.2
    ai, dist = find_nearest_neighbor(aj, agents)
    m = aj.velo
    if dist < max_visual_range:
        opinionDiff = abs(ai.opinion - aj.opinion)
        if (opinionDiff < aj.uncertainty):
            m = m + w*np.array([ai.velo[0], ai.velo[1]])
            #print("asdf")
            #print(opinionDiff)
            #print(aj.opinion)
            aj.opinion = aj.opinion + aj.conv * (ai.opinion - aj.opinion)
            #print(aj.opinion)
    if dist < 25:
        ma = np.array([posX - ai.getX(), posY - ai.getY()])
        m = m + 0.01 * ma
        label = 2
    m = normalize(m, 1)
    if posX - max_visual_range <= 0:
        d = 0.0 - (posX - max_visual_range)
        wd = d/max_visual_range
        m[0] = m[0] + wd * 1
        label = 3
    if posX + max_visual_range >= winWidth:
        d = (posX + max_visual_range) - winWidth
        wd = d/max_visual_range
        m[0] = m[0] + wd * -1
        label = 4
    if posY - max_visual_range <= 0:
        d = 0.0 - (posY - max_visual_range)
        wd = d/max_visual_range
        m[1] = m[1] + wd * 1
        label = 5
    if posY + max_visual_range >= winHeight:
        d = (posY + max_visual_range) - winHeight
        wd = d/max_visual_range
        m[1] = m[1] + wd * -1
        label = 6
    o = np.arctan2(m[1], m[0])
    aj.label = label 
    #aj.velo = m
    #aj.setOrientationO(o)
    return m, o

def find_neighbors_zoi(aj, zor_r, zoo_r, zoa_r, agents): 
    """K-nearest neighbor function for agents."""
    distances = [np.linalg.norm(np.array([aj.point.getCenter().getX(), aj.point.getCenter().getY()]) -
                                np.array([ai.point.getCenter().getX(), ai.point.getCenter().getY()])) for ai in agents]
    sorted_distances = np.sort(distances)
    nr = []
    no = []
    na = []
    for x in sorted_distances[1:]:
        if x <= zor_r:
            nr.append(agents[distances.index(x)])
        elif x <= zoo_r:
            no.append(agents[distances.index(x)])
        elif x <= zoa_r:
            na.append(agents[distances.index(x)])
        else:
            break
    return (nr, no, na)

def add_noise(vec, noise=2):
    """
    add noise to the vector
    """
    if noise == 0:
        return vec
    norm = np.linalg.norm(vec)
    random_angle = (np.random.rand()-0.5)*2*noise
    random_angle = np.radians(random_angle)
    vec_new = turn_vector(vec, random_angle)
    vec_new = normalize(vec_new, norm)
    return vec_new

# Couzin model
def couzin(aj, agents, winWidth, winHeight, max_visual_range):
    zor_r = 15
    zoo_r = 35
    zoa_r = 45
    max_visual_range = max_visual_range - 5
    m = aj.velo
    posX = aj.point.getCenter().getX()
    posY = aj.point.getCenter().getY()
    label = np.zeros((4))
    nr, no, na = find_neighbors_zoi(aj, zor_r, zoo_r, zoa_r, agents)
    ds = np.array([0.0, 0.0])
    if len(nr) > 0:
        for ai in nr:
            ma = np.array([ai.getX() - posX, ai.getY() - posY])
            ma = normalize(ma, 1)
            ds = ds - ma
        #ds = - normalize(ds, 1)
        label[0] = 1
    if len(na) > 0:
        for ai in na:
            ma = np.array([ai.getX() - posX, ai.getY() - posY])
            ma = normalize(ma, 1)
            ds = ds + ma
        #ds = normalize(ds, 1)
        label[2] = 1
    if len(no) > 0:
        for ai in no:
            ma = ai.velo
            ma = normalize(ma, 1)
            ds = ds + ma
        #ds = normalize(ds, 1)
        label[1] = 1
    if len(na) > 0 or len(no) > 0 or len(nr) > 0:
        ds = normalize(ds, 1)
    m = normalize(aj.velo + 0.1 * ds, 1)
    if posX - max_visual_range <= 0:
        d = 0.0 - (posX - max_visual_range)
        wd = d/max_visual_range
        m[0] = m[0] + wd * 1
        label[3] = 1
    if posX + max_visual_range >= winWidth:
        d = (posX + max_visual_range) - winWidth
        wd = d/max_visual_range
        m[0] = m[0] + wd * -1
        label[3] = 1
    if posY - max_visual_range <= 0:
        d = 0.0 - (posY - max_visual_range)
        wd = d/max_visual_range
        m[1] = m[1] + wd * 1
        label[3] = 1
    if posY + max_visual_range >= winHeight:
        d = (posY + max_visual_range) - winHeight
        wd = d/max_visual_range
        m[1] = m[1] + wd * -1
        label[3] = 1
    aj.label = label
    m = add_noise(m)
    o = np.arctan2(m[1], m[0])
    return m, o

#couzin with speed changes
c_vs = np.ones(29) * 0.2
def couzin_velomem(aj, agents, winWidth, winHeight, max_visual_range):
    t_steps = 40 #c_vs.shape
    zor_r = 15
    zoo_r = 35
    zoa_r = 45
    max_visual_range = max_visual_range - 5
    m = aj.velo
    posX = aj.point.getCenter().getX()
    posY = aj.point.getCenter().getY()
    label = np.zeros((4))
    nr, no, na = find_neighbors_zoi(aj, zor_r, zoo_r, zoa_r, agents)
    ds = np.array([0.0, 0.0])
    if len(nr) > 0:
        for ai in nr:
            aj.neighbours[ai.id] = t_steps
            ma = np.array([ai.getX() - posX, ai.getY() - posY])
            ma = normalize(ma, 1)
            ds = ds - ma
        #ds = - normalize(ds, 1)
        label[0] = 1
    if len(na) > 0:
        for ai in na:
            aj.neighbours[ai.id] = t_steps
            ma = np.array([ai.getX() - posX, ai.getY() - posY])
            ma = normalize(ma, 1)
            ds = ds + ma
        #ds = normalize(ds, 1)
        label[2] = 1
    if len(no) > 0:
        for ai in no:
            aj.neighbours[ai.id] = t_steps
            ma = ai.velo
            ma = normalize(ma, 1)
            ds = ds + ma
        #ds = normalize(ds, 1)
        label[1] = 1
    if label[1] == 1 or label[2] == 1 or label[0] == 1:
        ds = normalize(ds, 1)
        aj.future_vs[1:] = aj.future_vs[1:] + c_vs
        aj.neighbourocc.append(t_steps)
    velo = 1 + aj.future_vs[0]
    #print("velo :", velo)
    for i, value in enumerate(aj.neighbours):
        if value > 0:
            aj.neighbours[i] = value - 1
    for i, value in enumerate(aj.neighbourocc):
        if value > 0:
            aj.neighbourocc[i] = value - 1
    for i, value in reversed(list(enumerate(aj.neighbourocc))):
        if value < 1:
            del aj.neighbourocc[i]
    if velo-np.linalg.norm(aj.velo) > 1:
        print("velo: ", velo)
        print("aj.velo: ", aj.velo)
        print("acceleration: ", velo-np.linalg.norm(aj.velo))
    m = normalize(aj.velo + 0.1 * ds, velo)
    aj.future_vs = aj.future_vs
    if posX - max_visual_range <= 0:
        d = 0.0 - (posX - max_visual_range)
        wd = d/max_visual_range
        wd = wd * velo
        m[0] = m[0] + wd * 1
        label[3] = 1
    if posX + max_visual_range >= winWidth:
        d = (posX + max_visual_range) - winWidth
        wd = d/max_visual_range
        wd = wd * velo
        m[0] = m[0] + wd * -1
        label[3] = 1
    if posY - max_visual_range <= 0:
        d = 0.0 - (posY - max_visual_range)
        wd = d/max_visual_range
        wd = wd * velo
        m[1] = m[1] + wd * 1
        label[3] = 1
    if posY + max_visual_range >= winHeight:
        d = (posY + max_visual_range) - winHeight
        wd = d/max_visual_range
        wd = wd * velo
        m[1] = m[1] + wd * -1
        label[3] = 1
    aj.future_vs[:-1] = aj.future_vs[1:]
    aj.future_vs[-1] = 0
    aj.label = label
    m = add_noise(m)
    velolen = np.sqrt(m[0] * m[0] + m[1] * m[1])
    aj.velospeed = velolen
    o = np.arctan2(m[1], m[0])
    return m, o

#complex couzin
def couzin_mem_diff(aj, agents, winWidth, winHeight, max_visual_range):
    t_steps = 40
    zor_r = 15
    zoo_r = 35
    zoa_r = 45
    aj.repulsion_time = 0
    if np.sum(aj.neighbours > 0) > 3:
        zor_r = 45
        aj.repulsion_time = 1
    max_visual_range = max_visual_range - 5
    m = aj.velo
    posX = aj.point.getCenter().getX()
    posY = aj.point.getCenter().getY()
    label = np.zeros((4))
    nr, no, na = find_neighbors_zoi(aj, zor_r, zoo_r, zoa_r, agents)
    ds = np.array([0.0, 0.0])
    if len(nr) > 0:
        aj.neighbourocc.append(t_steps)
        aj.future_vs[1:] = aj.future_vs[1:] + c_vs
        for ai in nr:
            aj.neighbours[ai.id] = t_steps
            ma = np.array([ai.getX() - posX, ai.getY() - posY])
            ma = normalize(ma, 1)
            ds = ds + ma
        ds = - normalize(ds, 1)
        label[0] = 1
    else:
        if len(no) > 0: 
            for ai in no:
                aj.neighbours[ai.id] = t_steps
                if  aj.repulsion_time == 0:
                    ma = ai.velo
                    ma = normalize(ma, 1)
                    ds = ds + ma
            label[1] = 1
        if len(na) > 0:
            for ai in na:
                aj.neighbours[ai.id] = t_steps
                if  aj.repulsion_time == 0:
                    ma = np.array([ai.getX() - posX, ai.getY() - posY])
                    ma = normalize(ma, 1)
                    ds = ds + ma
            label[2] = 1
        if label[1] == 1 or label[2] == 1:
            if aj.repulsion_time == 0:
                ds = normalize(ds, 1)
            aj.future_vs[1:] = aj.future_vs[1:] + c_vs
            aj.neighbourocc.append(t_steps)
    if aj.repulsion_time == 0:
        velo = 1 + aj.future_vs[0]
    else:
        velo = np.linalg.norm(aj.velo)
    for i, value in enumerate(aj.neighbours):
        if value > 0:
            aj.neighbours[i] = value - 1
    for i, value in enumerate(aj.neighbourocc):
        if value > 0:
            aj.neighbourocc[i] = value - 1
    for i, value in reversed(list(enumerate(aj.neighbourocc))):
        if value < 1:
            del aj.neighbourocc[i]
    if velo-np.linalg.norm(aj.velo) > 0.15:
        velo = np.linalg.norm(aj.velo) + 0.15
    m = normalize(aj.velo + 0.2 * ds, velo)
    aj.future_vs = aj.future_vs
    if posX - max_visual_range <= 0:
        d = 0.0 - (posX - max_visual_range)
        wd = d/max_visual_range
        wd = wd * velo
        m[0] = m[0] + wd * 1
        label[3] = 1
    if posX + max_visual_range >= winWidth:
        d = (posX + max_visual_range) - winWidth
        wd = d/max_visual_range
        wd = wd * velo
        m[0] = m[0] + wd * -1
        label[3] = 1
    if posY - max_visual_range <= 0:
        d = 0.0 - (posY - max_visual_range)
        wd = d/max_visual_range
        wd = wd * velo
        m[1] = m[1] + wd * 1
        label[3] = 1
    if posY + max_visual_range >= winHeight:
        d = (posY + max_visual_range) - winHeight
        wd = d/max_visual_range
        wd = wd * velo
        m[1] = m[1] + wd * -1
        label[3] = 1
    aj.future_vs[:-1] = aj.future_vs[1:]
    aj.future_vs[-1] = 0
    aj.label = label
    m = add_noise(m)
    velolen = np.linalg.norm(m)
    if aj.repulsion_time > 0:
        if aj.velospeed > 1:
            velolen = max(0.5, velolen-0.15)
            m = normalize(m, velolen)
    aj.velospeed = velolen
    o = np.arctan2(m[1], m[0])
    return m, o


#complex couzin OLD OLD OLD
def couzin_mem_diff_OLD(aj, agents, winWidth, winHeight, max_visual_range):
    t_steps = 40 #c_vs.shape
    #print(np.sum(aj.neighbours > 0))
    zor_r = 15
    zoo_r = 35
    zoa_r = 45
    aj.repulsion_time = 0
    # if we had at least 4 distinct labors in last n steps: repulsion time
    if np.sum(aj.neighbours > 0) > 3:
        zor_r = 45
        aj.repulsion_time = 1
    max_visual_range = max_visual_range - 5
    m = aj.velo
    posX = aj.point.getCenter().getX()
    posY = aj.point.getCenter().getY()
    label = np.zeros((4))
    nr, no, na = find_neighbors_zoi(aj, zor_r, zoo_r, zoa_r, agents)
    ds = np.array([0.0, 0.0])
    #we only consider Repulsion Zone if agents are inside repulsion sides
    if len(nr) > 0:
        aj.neighbourocc.append(t_steps)
        aj.future_vs[1:] = aj.future_vs[1:] + c_vs
        for ai in nr:
            aj.neighbours[ai.id] = t_steps
            ma = np.array([ai.getX() - posX, ai.getY() - posY])
            ma = normalize(ma, 1)
            ds = ds + ma
        ds = - normalize(ds, 1)
        label[0] = 1
    else:
        if len(no) > 0: 
            for ai in no:
                aj.neighbours[ai.id] = t_steps
                if  aj.repulsion_time == 0:
                    ma = ai.velo
                    ma = normalize(ma, 1)
                    ds = ds + ma
            label[1] = 1
        if len(na) > 0:
            for ai in na:
                aj.neighbours[ai.id] = t_steps
                if  aj.repulsion_time == 0:
                    ma = np.array([ai.getX() - posX, ai.getY() - posY])
                    ma = normalize(ma, 1)
                    ds = ds + ma
            label[2] = 1
        if label[1] == 1 or label[2] == 1:
            if aj.repulsion_time == 0:
                ds = normalize(ds, 1)
            aj.future_vs[1:] = aj.future_vs[1:] + c_vs
            aj.neighbourocc.append(t_steps)
    velo = 1 + aj.future_vs[0]
    for i, value in enumerate(aj.neighbours):
        if value > 0:
            aj.neighbours[i] = value - 1
    for i, value in enumerate(aj.neighbourocc):
        if value > 0:
            aj.neighbourocc[i] = value - 1
    for i, value in reversed(list(enumerate(aj.neighbourocc))):
        if value < 1:
            del aj.neighbourocc[i]
    m = normalize(aj.velo + 0.2 * ds, velo)
    aj.future_vs = aj.future_vs
    if posX - max_visual_range <= 0:
        d = 0.0 - (posX - max_visual_range)
        wd = d/max_visual_range
        wd = wd * velo
        m[0] = m[0] + wd * 1
        label[3] = 1
    if posX + max_visual_range >= winWidth:
        d = (posX + max_visual_range) - winWidth
        wd = d/max_visual_range
        wd = wd * velo
        m[0] = m[0] + wd * -1
        label[3] = 1
    if posY - max_visual_range <= 0:
        d = 0.0 - (posY - max_visual_range)
        wd = d/max_visual_range
        wd = wd * velo
        m[1] = m[1] + wd * 1
        label[3] = 1
    if posY + max_visual_range >= winHeight:
        d = (posY + max_visual_range) - winHeight
        wd = d/max_visual_range
        wd = wd * velo
        m[1] = m[1] + wd * -1
        label[3] = 1
    aj.future_vs[:-1] = aj.future_vs[1:]
    aj.future_vs[-1] = 0
    aj.label = label
    m = add_noise(m)
    velolen = np.sqrt(m[0] * m[0] + m[1] * m[1]) 
    if aj.repulsion_time > 0:
        if aj.velospeed > 1:
            velolen = max(0.5, velolen-1)
            m = normalize(m, velolen)
            if velolen < 1:
                aj.repulsion_time = 0
    
    aj.velospeed = velolen
    o = np.arctan2(m[1], m[0])
    return m, o


###################################### Movement Models END ######################################

def run_complexrnn(aj, num_features, m, o):
    #rnn.zero_grad()
    global Y_HIST
    windows = 10
    X = (np.asarray(aj.history[-windows:]))
    X = X[:, 6+4:]
    dx = np.zeros((2, windows, num_features))
    dx[0, :, :] = X
    dx[1, :, :] = X
    X = dx.astype(np.float32)
    X = torch.from_numpy(X)
    X = torch.autograd.Variable(X).cuda()
    Y, hidden = rnn.forward(X, verbose=False)
    pi, sigma, mu = Y 
    Y = getMeanSample(pi, sigma, mu) 
    Y = MDN.sample(pi, sigma, mu)
    Y = Y[0, :].cpu().data.numpy()
    dx = Y[0]
    dy = Y[1]
    dxt, dyt = dx, dy
    aj.velo = np.array([dxt, dyt]) 
    newOrientation = np.arctan2(Y[3], Y[2]) 
    if newOrientation > np.pi:
        newOrientation = - np.pi + (newOrientation - np.pi)
    if newOrientation < -np.pi:
        newOrientation =  np.pi - (- np.pi - newOrientation)
    aj.setOrientationO(newOrientation)
    aj.setOrientationO(np.arctan2(dy, dx))


    
def assimilate_velott(aj, agents):
    """Calculate the new velocity for a single agent by looking at the velocity of the nearest neighbour."""
    oldO = aj.orientation
    oldm = aj.velo
    w = 0.2
    ai = find_nearest_neighbor(aj, agents)
    m = aj.velo + w*np.array([ai.velo[0], ai.velo[1]])
    m = normalize(m, 1)
    o = np.arctan2(m[0], m[1])
    dm = m
    do = angle_between_rad(o, oldO)
    dxo, dyo = np.cos(do), np.sin(do)
    dxt, dyt = transformCordinateSystem(oldO, dm[0], dm[1])
    # dxt, dyt, dxo, dyo entspricht perfekter vorhersage
    
    dx, dy = transformCordinateSystem(-oldO, dxt, dyt)
    aj.velo = np.array([dx, dy]) #normalize(np.array([dxt, dyt]), 2) 
    newOrientation = oldO + np.arctan2(dyo, dxo)
    if newOrientation > np.pi:
        newOrientation = - np.pi + (newOrientation - np.pi)
    if newOrientation < -np.pi:
        newOrientation =  np.pi - (- np.pi - newOrientation)
    aj.setOrientationO(newOrientation)  
    


def find_nearest_neighbor(aj, agents): 
    """Nearest neighbor for agents."""
    distances = [np.linalg.norm(np.array([aj.point.getCenter().getX(), aj.point.getCenter().getY()]) -
                                np.array([ai.point.getCenter().getX(), ai.point.getCenter().getY()])) for ai in agents]
    distances[distances.index(0.0)] = 99999999999999999
    return agents[distances.index(min(distances))],  distances[distances.index(min(distances))]

def find_k_nearest_neighbor(aj, k, agents): 
    """K-nearest neighbor function for agents."""
    distances = [np.linalg.norm(np.array([aj.point.getCenter().getX(), aj.point.getCenter().getY()]) -
                                np.array([ai.point.getCenter().getX(), ai.point.getCenter().getY()])) for ai in agents]
    sorted_distances = np.sort(distances)
    neighbours = []
    for x in sorted_distances[1:k+1]:
        neighbours.append(agents[distances.index(x)])
    return (neighbours)


Y_HIST = np.zeros((1, 4))
def run_bee_net(aj, num_features):
    rnn.zero_grad()
    global Y_HIST
    windows = 2
    X = (np.asarray(aj.history[-windows:]))
    X = X[:, 6:]
    print("X: ", X[:, :5])
    dx = np.ones((2, windows, num_features)) * 0.333
    dx[0, :, :] = X
    dx[1, :, :] = X
    X = dx.astype(np.float32)
    X = torch.from_numpy(X)
    X = torch.autograd.Variable(X).cuda()
    Y, hidden = rnn.forward(X, verbose=False)
    pi, sigma, mu = Y 
    Y = MDN.sample(pi, sigma, mu)
    Y = Y[0, :].cpu().data.numpy()
    Y_HIST.append(Y)
    print("Prediction: ", Y)
    dx = Y[0]
    dy = Y[1]
    dxt, dyt = dx, dy
    aj.velo = np.array([dxt, dyt]) 
    newOrientation = np.arctan2(Y[3], Y[2]) #aj.orientation + np.arctan2(Y[3], Y[2])
    if newOrientation > np.pi:
        newOrientation = - np.pi + (newOrientation - np.pi)
    if newOrientation < -np.pi:
        newOrientation =  np.pi - (- np.pi - newOrientation)
    aj.setOrientationO(newOrientation)


    
def saveNetworkTimestep(t_save, network_historys):
    ts = np.ones((historys.shape[0], 1, 3)) * -1000
    for ai in agents:  
        x = ai.getX()
        y = ai.getY()
        o = ai.orientation
        ts[ai.id, 0] = x, y, o
    network_historys = np.concatenate((network_historys, ts), 1)
    print(network_historys.shape)
    return network_historys
    
def avoid_border_crossing(aj, winWidth, winHeight):
    if aj.point.getCenter().getX() + aj.velo[0] <= 0 or aj.point.getCenter().getX() + aj.velo[0] >= winWidth:
        aj.velo[0] = aj.velo[0]*(-1)
        print("X CROSS")
    if aj.point.getCenter().getY() + aj.velo[1] <= 0 or aj.point.getCenter().getY() + aj.velo[1] >= winHeight:
        aj.velo[1] = aj.velo[1]*(-1)
        print("Y CROSS")
    
def update_agents(algorithm, window, winWidth, winHeight, draw, switch, OSHistory, vision_bins, max_visual_range):
    global network_historys
    num_features = 5 + vision_bins * 8 #8
    if timestep == switch and algorithm == 7:
        agents[:] = [aj for aj in agents if not len(aj.history) < 34]
        draw_agents(window, draw)
        network_historys = np.ones(((historys.shape[0], 0, 3))) * (-1000)
        #time.sleep(10)
    for aj in agents:
        if algorithm == 0:
            if timestep < switch:
                simple_step(aj, agents)
            else:
                m, o = constant_step(aj, agents, winWidth, winHeight, max_visual_range)
                aj.velo = m
                aj.setOrientationO(o)
        elif algorithm == 1:
            if timestep < switch:
                simple_step(aj, agents)
            else:
                m, o = couzin(aj, agents, winWidth, winHeight, max_visual_range)
                aj.velo = m
                aj.setOrientationO(o)
        elif algorithm == 2:
            if timestep < switch:
                simple_step(aj, agents)
            else:
                m, o = couzin_velomem(aj, agents, winWidth, winHeight, max_visual_range)
                aj.velo = m
                aj.setOrientationO(o)
        elif algorithm == 3:
            window_step(aj)
            time.sleep(0.01)
        elif algorithm == 4:
            if timestep < switch:
                simple_step(aj, agents)
            else:
                m, o = couzin(aj, agents, winWidth, winHeight, max_visual_range)
                run_complexrnn(aj, num_features, m, o)        
        elif algorithm == 5:
            if timestep < switch:
                simple_step(aj, agents)
            else:
                m, o = couzin_mem_diff(aj, agents, winWidth, winHeight, max_visual_range)
                aj.velo = m
                aj.setOrientationO(o)

        elif algorithm == 6:
            if timestep < switch:
                simple_step(aj, agents)
            else:
                m, o = couzin_mem_diff(aj, agents, winWidth, winHeight, max_visual_range)
                run_complexrnn(aj, num_features, m, o)
        elif algorithm == 9:
            if timestep < switch:
                real_step(aj, agents, window)
            else:
                m, o = couzin(aj, agents, winWidth, winHeight, max_visual_range)
                run_complexrnn(aj, num_features, m, o)
        else:
            print("No algorithm specified")
            return
    if timestep >= switch and algorithm == 7:
        network_historys = saveNetworkTimestep(timestep-switch, network_historys)
        
        
def update_vision(max_visual_range, vision_bins, borders, OSHistory, verbose = True):
    positions = np.asarray([[ai.getX(), ai.getY(), ai] for ai in agents])
    neighbours= spatial.cKDTree(positions[:,:2]).query_ball_point(positions[:,:2], max_visual_range)
    for j, pos in enumerate(positions):
        currstate = pos[2].getCurrState(vision_bins)
        OSHistory.save(currstate[4], currstate)
    for j, pos in enumerate(positions):
        if (positions[j,0]<0 or positions[j, 1]<0):
            pos[2].vision = np.zeros(vision_bins*8)
        else:
            cur_neighbours = [x for x in neighbours[j] if x != j] 
            neighbour_positions = positions[cur_neighbours]
            neighbour_positions = np.asarray([n.getCurrState(vision_bins) for n in neighbour_positions[:, 2]])
            beesurr = getBeeSurroundings(pos[2].getCurrState(vision_bins), neighbour_positions, OSHistory, max_visual_range, vision_bins, verbose = verbose, borders = borders)
            pos[2].vision = beesurr
            
def update_vision_wall(max_visual_range, vision_bins, borders, OSHistory, verbose = True):
    positions = np.asarray([[ai.getX(), ai.getY(), ai] for ai in agents])
    for j, pos in enumerate(positions):
        currstate = pos[2].getCurrState(vision_bins)
        OSHistory.save(currstate[4], currstate)
    for j, pos in enumerate(positions):
        if (positions[j,0]<0 or positions[j, 1]<0):
            pos[2].vision = np.zeros(vision_bins*8)
        else:
            beesurr = getBeeSurroundings(pos[2].getCurrState(vision_bins), [], OSHistory, max_visual_range, vision_bins, verbose = verbose, borders = borders)
            pos[2].vision = beesurr
            
#bins_distance, bins_SINorientation, bins_COSorientation, bins_activity, 
#bins_binary, bins_relativeOsin, bins_relativeOcos, borders
def decode_beeSurroundings_wall(aj, vision, max_visual_range, window, num_bins = 36, borders=[[0,0], [370, 370]]):
    """
    Seperate environment of an agent into bins and find nearest neighbour for all bins.
    Add orientation and activity of the neighbour bees.
    """
    posX = aj.getX()
    posY = aj.getY()
    orientation = aj.orientation
    bin_distance = 360/num_bins
    angles = np.linspace(-180+bin_distance/2, 180-bin_distance/2, num_bins, endpoint=True)
    print("a", angles.shape)
    angles = np.radians(angles)
    ovec = np.array([np.sin(orientation), np.cos(orientation)])
    target_dirs = [normalize(turn_vector(ovec, angle), max_visual_range) for angle in angles]
    #borders = get_walls_in_sight(np.array([posX, posY]), orientation, max_visual_range, num_bins, borders)
    wall_borders = vision[7*num_bins:8*num_bins]
    print("x, y: ", posX, ", ", posY)
    print("WB", wall_borders)
    print("wb", wall_borders.shape)
    for i, direction in enumerate(target_dirs):
        print(i)
        aj.wlines[i].undraw()
        aj.wlines[i] = Line(aj.point.getCenter(), Point(aj.getX() + direction[0], aj.getY() + direction[1]))
        aj.wlines[i].setFill('blue')
        aj.wlines[i].draw(window)
        if wall_borders[i] > 0.0:
            aj.wlines[i].setFill('green')
            print("i, direction", i, ", ", direction)



def decode_beeSurroundings(aj, vision, max_visual_range, window, num_bins = 36, borders=[[0,0], [370, 370]]):
    posX = aj.getX()
    posY = aj.getY()
    orientation = aj.orientation
    bins_distance = vision[0:num_bins]
    bins_SINorientation = vision[num_bins:2*num_bins]
    bins_COSorientation = vision[2*num_bins:3*num_bins]
    bins_activity = vision[3*num_bins:4*num_bins]
    bins_binary = vision[4*num_bins:5*num_bins]
    print("--------------------------")
    print("X, Y, o: ", posX, posY, orientation)
    print("coso, sino", np.cos(orientation), np.sin(orientation))
    print("bins_binary: ", bins_binary) # 5
    print("bins_sino: ", bins_SINorientation) # 5
    print("bins_coso: ", bins_COSorientation) # 5
    print("bins_activity: ", bins_activity) # 5
    angle_borders = np.linspace(-180, 180, num_bins + 1)
    angles = np.radians(angle_borders)
    ovec = np.array([np.cos(orientation), np.sin(orientation)])
    target_dirs = [normalize(turn_vector(ovec, -angle), max_visual_range) for angle in angles] #6 -2*np.pi/5
    #print(target_dirs)
    bin_distance = 360/num_bins
    ds = np.zeros(num_bins+1) # 6
    for i in range(0, num_bins):
        if bins_binary[i] > 0.5:
            ds[i] = bins_distance[i] * max_visual_range
            ds[i+1] =bins_distance[i] * max_visual_range #i +i+1 must be drawn
            
    #bins_distance[i] * max_visual_range= max_visual_range - np.linalg.norm(sight_vector)
    if ds[0]< ds[-1]:
        ds[0] = ds[-1]
    else:
        ds[-1] = ds[0]
    #ds[0] = 1
    #ds[-1] = 1
    print("ds", ds)
    for i, direction in enumerate(target_dirs):
        direction = normalize(direction, 50) #ds[i])
        aj.vlines[i].undraw()
        aj.vlines[i] = Line(aj.point.getCenter(), Point(aj.getX() + direction[0], aj.getY() + direction[1]))
        aj.vlines[i].setFill('black')
        aj.vlines[i].draw(window)
        if ds[i] > 0.0:
            aj.vlines[i].setFill('red')
        #    print("i, direction", i, ", ", direction)
        #bins[i] = max(0, 1 - np.linalg.norm(position-intersection_point)/radius)

def normalize(vec, norm):
    """Normalizes the vector and multiplies it with norm."""
    if np.linalg.norm(vec) == 0:
        print("normalize with zero vector")
        return vec
    return(norm*vec/np.linalg.norm(vec))


def update_vision_z(max_visual_range, vision_bins, OSHistory, verbose = True):
    for ai in agents:
        ai.vision = np.zeros(8*vision_bins)
            
### Simulation
def do_simulation(num_agents, winWidth, winHeight, num_timesteps, algorithm, vision_bins, max_visual_range, switch, OSHistory, draw = True, save_positions=False):
    borders = [[0,0], [winWidth, winHeight]]
    global timestep
    timestep = 0
    win = 0
    if draw == True:
        win = GraphWin("Window", winWidth, winHeight, autoflush=False)
        win.setBackground('white')
    initialize(num_agents, win, winWidth, winHeight, vision_bins, draw, algorithm)
    for i in range(num_timesteps):
        #time.sleep(0.2)
        if draw == True:
            print("Timesteps: ", i)
        if timestep < 2:
            #time.sleep(2)
            update_vision(max_visual_range, vision_bins, borders, OSHistory, verbose = False)
            #update_vision_wall(max_visual_range, vision_bins, borders, OSHistory, verbose = False)
        else:
            update_vision(max_visual_range, vision_bins, borders, OSHistory, verbose = True)
            #update_vision_wall(max_visual_range, vision_bins, borders, OSHistory, verbose = True)
        update_agents(algorithm, win, winWidth, winHeight, draw, switch, OSHistory, vision_bins, max_visual_range)
        if timestep > 5:
            for ai in agents:
                ai.appendTimestep(OSHistory, vision_bins)
        for ai in agents:
            avoid_border_crossing(ai, winWidth, winHeight)
            move_agent(ai, win, draw, algorithm)
        if False:
            for aj in agents:
                decode_beeSurroundings(aj, aj.vision, 100, win, vision_bins, borders)
                #decode_beeSurroundings_wall(aj, aj.vision, 100, win, vision_bins, borders)
        if draw == True:
            win.update()
        #time.sleep(0.1)
        timestep = timestep + 1
    if draw == True:
        win.close()
        win.update()
    return agents

In [ ]:
VISION -> APPEND TIMESTEP(labels saved) -> Move_AGENTS(Labels generated) -> Vision
VISION_T Data_T Label_T


In [ ]:
c_vs = np.array([0.01, 0.01, 0.02, 0.02, 0.03, 0.03, 0.04, 0.04, 0.05, 0.05, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.05, 0.05, 0.04, 0.04, 0.03, 0.03, 0.02, 0.02, 0.01, 0.01])
c_vs = c_vs * 3
c_vs

In [ ]:
c_vs = np.zeros(39)
c_vs

In [ ]:
c_vs = np.array([0.1, 0.1, 0.2, 0.2, 0.3, 0.3, 0.4, 0.4, 0.5, 0.6, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8,
       0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.6, 0.5, 0.5, 0.5, 0.4, 0.4, 0.3, 0.3, 0.2, 0.2, 0.1, 0.1])
c_vs = c_vs * 1
c_vs

In [ ]:
#rnn = SimpleRNN(n_features=5 + (vision_bins * 8), n_outputs=4).cuda()
rnn = SimpleRNN(n_features=5 +(vision_bins * 8), n_outputs=4).cuda()
#rnn = torch.load('simplemodel-rnn.pt')
#rnn = torch.load('couzinmodel-rnn.pt')
#rnn = torch.load('couzinvelomodel-rnn.pt')
#rnn = torch.load('complexcouzin-rnn.pt')
rnn = torch.load('bee40-rnn.pt')
optimizer = torch.optim.RMSprop(rnn.parameters(), lr=0.0001)
running_loss = 0.0
#criterion = torch.nn.SmoothL1Loss()
#criterion.cuda(

In [ ]:
rec_losses = []

import torch
winWidth = 550
winHeight = 550
T = 2500
N = 30
track_ids = np.arange(N)
vision_bins = 16
max_visual_range = 50
l = len(track_ids)
for i in range(0, l):
    track_ids[i] = i
num_features = 5 + (vision_bins * 8)
OSHistory = OneStepBeeHistory(5 +2+(vision_bins * 8), track_ids)
#scale = 2
#rnn = torch.load("complexrnn.pt")
#rnn.eval() # evaluation mode
algorithm = 3 #5
#Attention: If bee RNN is run the visual range must be 20
#0: Simple Model             1: Simple Couzin
#2: Couzin Velo              3: windows step 
#4: RNN                      5: Complex Couzin
#6: RNN for CC
#9: Init with real data and run real data.
switch = 50
start = time.time()
do_simulation(N, winWidth, winHeight, T, algorithm, vision_bins, max_visual_range, switch, OSHistory,draw = True)
end = time.time()
print("Time in seconds: ", end - start)


In [ ]:
import warnings
warnings.filterwarnings('error')
warnings.filterwarnings('ignore')
rec_losses = []
def generate_train_data(runs):
    start = time.time()
    vision_bins = 16
    max_visual_range = 50
    window_size = 40
    img_dtype = tables.Float64Atom()  # dtype in which the images will be saved
    data_shape = (0, window_size -1, 5 + 6 + 4 + (vision_bins * 8))
    #data_shape = (0, window_size -1, 5 + 6 + (vision_bins*1))
    with tables.open_file('pathtofile.h5', mode='a') as hdf5_file:
        #hdf5_file.create_earray(hdf5_file.root, 'train', img_dtype, shape = data_shape)
        print(hdf5_file.root.train.shape)
        for j in range(0, runs):
            print(j)
            N = random.randint(1, 35)
            track_ids = np.arange(N)
            for i in range(0, N):
                track_ids[i] = i
            OSHistory = OneStepBeeHistory(5 + 2 + (vision_bins * 8), track_ids)
            T = random.randint(300, 1200)
            switch = 5
            algorithm = 5
            W = random.randint(350, 550)
            winWidth, winHeight = W, W
            do_simulation(N, winWidth, winHeight, T, algorithm, vision_bins, max_visual_range, switch, OSHistory, draw = False)
            for aj in agents:
                track = np.asarray(aj.history)
                for i in range(35, len(track)-window_size, 2):
                    #print(track[None, i: i+window_size -1].shape)
                    hdf5_file.root.train.append(track[None, i: i+window_size -1])
    end = time.time()
    print("Time in seconds: ", end - start)
    hdf5_file.close()    
generate_train_data(215)

In [ ]:
10# Random shuffle the training data on the hard disk, for large data sets, this takes ages
import random
import time
import tables
import numpy as np
start = time.time()
hdf5_file = tables.open_file('pathtofile.h5', mode='a')
print(hdf5_file.root.train.shape)
print(hdf5_file.root.train[0:10, :, :])
random.shuffle(hdf5_file.root.train)
print(hdf5_file.root.train.shape)
print(hdf5_file.root.train[0:10, :, :])
hdf5_file.close()
end = time.time()
hdf5_file.close()
print("Time in seconds for shuffle: ", end - start)